# What do we want?
```
... we all just wanna be big rockstars
And live in hilltop houses driving 15 cars...

``` 
® Nickelback


Training neural network needs the following:

1. Understand which input and output you have
2. Make dataset for the training
3. Optimize training process
4. Analyse output

## How to produce training data

1. dd4hep simulation
2. Marlin pre-analysis
3. Convert file with ntuple, trees to root files with data(pair image - fem) 
4. Do it on cluster

### Fisrt two steps

You need build all software(LCIO, dd4hep...) or mount it. You are lucker, it has already done.


```
at ijclab: 

log in to ssh-centos3 or ssh-centos2
> bash 
cp all under /users/flc/poeschl/ild to one of your directories
go to the sub-directory "ild"
modify in init_ilcsoft.sh 
export MARLIN_DLL="$MARLIN_DLL:/users/flc/poeschl/ild/ildana/v01-13/lib/libLCTuple.so.1.13.0"
to
export MARLIN_DLL="$MARLIN_DLL:<yourdirectory>/ild/ildana/v01-13/lib/libLCTuple.so.1.13.0"
> . ./init_ilcsoft.sh
--> We use a pre-compiled version of ilcsoft that can be found in the directory
/cvmfs/ilc.desy.de/sw/x86_64_gcc82_centos7/v02-02-03 
The directory sits physically at DESY but is mounted over the network to the ICJLab machines. Since Ukrainian institutes are member of e.g. CERN collaborations (and also of ILD) cvmfs mounting should be available "somewhere" in Ukraine. 
> ls -al
--> You'll find below two directories called ildsim and ildana
> cd ildsim
> cd ildsim/v02-02-03/StandardConfig/production
Simulation of events
> ddsim \
  --outputFile yoursimoutput.slcio \
  --compactFile $lcgeo_DIR/ILD/compact/ILD_l5_v02/ILD_l5_v02.xml \
  --steeringFile ddsim_steer.py

ddsim_steer.py is a steering file for the simulation of individual particles
There is a number of parameters but for the time being the most important ones are
SIM.numberOfEvents = 100  --> Number of events
SIM.gun.direction = (0, 1, 0) --> The direction vector of the particle. in this case in y-direction
SIM.gun.particle = "pi-" --> The particle type, here "pi-", for electrons you'd put "e-"
SIM.gun.position = (0.0, 1800.0, 100.0) --> x,y,z position of starting point of particle (here about in front of the calorimeter, may need fine tuning)

Reconstruction of simulated events
Marlin MarlinStdReco.xml \
  --constant.lcgeo_DIR=$lcgeo_DIR \
  --constant.DetectorModel=ILD_l5_o1_v02 \
  --constant.OutputBaseName=yourrecoutput \
  --global.LCIOInputFiles=yoursimoutput.slcio

among others you'll produce a file called yourrecoutput_REC.slcio

Now go to ildana.myana

> Marlin --global.LCIOInputFiles=<yourdirectory>/ild/ildsim/v02-02-03/StandardConfig/production/yourrecoutput_REC.slcio --global.MaxRecordNumber=1000 MarlinStdRecoLCTuple.xml

This will create a root file that looks like the ones your are working with. The name is StandardReco_LCTuple.root and your can rename it to a more reasonable name. 

In principle your should now also look at the event display using "glced" but this would be difficult from remote.  

```

### Third step

The code which is capable to do it locates on [github](https://github.com/Padniuk/XCalSimulation)

### Openstack

A Cluster which we use you can find [here](https://horizon.ijclab.in2p3.fr/) 


How to enter?

```bash
ssh -i "/path/to/key.pem" username@123.45.12.123
```

*init_software.sh* - script to mount dd4hep and other "fiends"

*/mnt* - folder which contains mounted volumes in

How to mount?

```bash
sudo mount /dev/vdd /mnt/lctuple/
sudo mount /dev/vdc /mnt/simulation/
sudo mount /dev/vdb /mnt/marlin/
```

### Let's simulate!

There is some [code](https://github.com/yuichiok/SiWECAL-Sim/blob/main/generation/run_scripts/send_condor.py) which I used as a base

But such a implementation is better(I did not tested it on our case): 

In [ ]:
from multiprocessing import Pool

number_of_threads = 5

def find_sum(a):
    sum_=0
    for i in range(1000000000):
        sum_+=i
    return sum_+a


with Pool(number_of_threads) as pool:
    result = pool.map(find_sum, [i for i in range(8)])

    for i, res in enumerate(result):
        with open(f"output_{i}.txt", "a") as f:
            f.write(str(res)+"\n")

### Marlin

Run *marlin_analysis.sh* to pre-process data

### LCTuple

I was forced to redo LCTuple code and change path to LCTuple soft in *init_software.sh* because default version does not provide informaion about hit contributors.

Compiled new version is already located inside an instance.

### Create a image - $f_{em}$ pairs

I developed [own code](https://github.com/Padniuk/XCalSimulation) for such a purpose. You can fork it, or snipped.

### How to access to my instance?

Make a user which will have corresponding permissions


## Training

I was training network on my own machine, because ruche had some troubles. But I know the main ideas on how to do it:

1. [Documentation](https://mesocentre.pages.centralesupelec.fr/user_doc/ruche/01_cluster_overview/)
2. [Examples](https://gitlab-research.centralesupelec.fr/mesocentre-public/ruche_examples/)